In [1]:
import pandas as pd
from opencc import OpenCC
from tqdm import tqdm
tqdm.pandas()

cc = OpenCC('s2t')


df = pd.read_excel('PDCS - 20220101 to 20241203.xlsx')

df = df[['bill_no', 'group_line', 'manufacture_time', 'exception_desc', 'defects_name',
   'reason_fix', 'qc_suggestion']]

# 確保所有值都是字串，並將 NaN 或空值轉換為空字串
df['reason_fix'] = df['reason_fix'].fillna('').astype(str)
df['exception_desc'] = df['exception_desc'].fillna('').astype(str)


df['corpus'] = df['reason_fix'] + ' '+df['exception_desc']
df['corpus'] = df['corpus'].progress_apply(lambda x: cc.convert(x))

100%|█████████████████████████████████████| 7329/7329 [00:02<00:00, 2542.45it/s]


In [2]:
import re

def clean_text(text):
    # 移除 _x000D_
    text = re.sub(r'_x000D_', '', text)
    # 替換換行符號 (\n) 為空格
    text = re.sub(r'\n', ' ', text)
    # 替換多個空格為單個空格
    text = re.sub(r'\s+', ' ', text)
    # 移除前後空白字符
    return text.strip()

# 應用清洗函數到 'reason_fix' 欄位
df['corpus'] = df['corpus'].apply(clean_text)

In [152]:
synonym_df

,main,branch1
0,貼裝頭,貼片頭
1,Feeder,"飛達, feeder"
2,吸嘴,nozzle
3,殘錫,"沾錫, 纏錫, 馋錫"
4,軌道,"滑軌, 導軌"
5,端板,"拿板, 取板"
6,機台,治具
7,殘膠,"沾膠, 纏膠, 馋膠"


In [3]:
synonym_df = pd.read_excel('SMT同義字整理.xlsx')

def synonym_replacement(input_text, synonym_df=synonym_df):
    """
    Replace words in the input text with their corresponding synonyms based on the synonym dataframe.

    Args:
        input_text (str): The text to be processed.
        synonym_df (pd.DataFrame): A dataframe with two columns: 'main' and 'branch1'.
                                  'main' contains the target word,
                                  'branch1' contains a comma-separated string of synonyms for the target word.

    Returns:
        str: The processed text with synonyms replaced.
    """
    replacement_dict = {}
    for _, row in synonym_df.iterrows():
        main = row['main']
        synonyms = row['branch1'].split(', ')
        for synonym in synonyms:
            replacement_dict[synonym] = main

    # Replace synonyms with main terms
    for synonym, main in replacement_dict.items():
        input_text = input_text.replace(synonym, main)

    return input_text

synonym_replacement('該群組的feeder有沾膠現象')

'該群組的Feeder有殘膠現象'

In [4]:
df['corpus'] = df['corpus'].apply(synonym_replacement)

In [5]:
df.loc[0,'corpus']

'1.現場確認為電感偏位不良，3顆用量，3個位置均有不良產出； 2.查詢不良板路由時間：1/7-1/8白晚班均有不良產出，目前不良持續中； 3.更換其它廠商（EATON）交叉驗證，未有不良產出，通過爐前觀察，發現部分流線主板出現電感歪斜不良，仔細目檢電感焊接端子，物料底部兩端凸材面不平整； 4.取下不良板上物料，在顯微鏡下觀察：底部殘錫不均，導致過爐時兩端受力不均，出現偏位不良； 綜上：初步嫌疑為電感底部凸材不均，導致過爐異常，請SQE知會廠商改善。 臨時對策： 1.知會生技持續優化貼裝坐標； 2.更換其它廠商使用； S33線在生產SIERRA-V2（M10314-003）機種時，PL415/PL2005/PL2305位置電感偏位，不良率：28.57%（16/56），請相關單位協助分析改善！'

In [6]:
#df = df[:500]

## 清理文本並進行分詞

In [7]:
import jieba

# 读取自定义词典
with open('關鍵字熱詞.txt', 'r', encoding='utf-8') as f:
    custom_words = f.readlines()

# 去除多余的换行符并处理词频和词性
processed_words = [f"{word.strip()} 10000 n" for word in custom_words if word.strip()]

# 将处理后的内容保存为新的词典文件
with open('processed_關鍵字熱詞.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(processed_words))

# 加載自定義詞典
jieba.load_userdict('processed_關鍵字熱詞.txt')


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.498 seconds.
Prefix dict has been built successfully.


In [8]:
import jieba.posseg as pseg
import re
import pandas as pd
from multiprocessing import Pool

# 清理文本函數
def clean_and_segment(text):
    if not isinstance(text, str):  # 確保文本是字串
        return ''
    text = re.sub(r'[^\u4e00-\u9fffa-zA-Z0-9]+', ' ', text)  # 僅保留中文、英文、數字
    words = pseg.cut(text)  # 使用 jieba.posseg 進行分詞與詞性標註
    return " ".join([word for word, flag in words if flag in ['n', 'nz', 'nt', 'nl', 'eng', 'x']])  # 保留名詞和專有名詞



# n：普通名詞
# 代表一般的名詞，例如「物料」、「廠商」、「現場」。
# nz：專有名詞
# 代表專有名詞，例如「SMT」、「Feeder」、「Lenovo」。
# nt：機構團體名
# 代表組織或機構的名稱，例如「公司」、「工廠」。
# nl：地名
# 代表地點或地名，例如「深圳」、「台北」。
# eng：英文單詞
# 代表英文字詞，例如「Rail」、「pin」、「ICT」。

# v：動詞，例如「操作」、「檢查」。
# a：形容詞，例如「不良」、「快速」。
# r：代詞，例如「我們」、「這個」。
# m：數量詞，例如「一個」、「三次」。
# d：副詞，例如「非常」、「嚴重」。

# 清理並分詞
#df['cleaned'] = df['reason_fix'].progress_apply(clean_and_segment)



# 多進程分詞函數
def parallel_segment(df, column, num_processes=4):
    with Pool(num_processes) as pool:
        # 將指定欄位的文本分配到多進程中
        results = pool.map(clean_and_segment, df[column])
    return results


# 使用多進程加速清理和分詞
df['cleaned'] = parallel_segment(df, 'corpus', num_processes=8)

# 3Min

In [10]:
# import jieba
# import jieba.posseg as pseg
# import re

# # 加載自定義詞典
# jieba.load_userdict('關鍵字熱詞.txt')


# # 清理文本函數
# def clean_and_segment(text):
#     if not isinstance(text, str):  # 確保文本是字串
#         return ''
#     text = re.sub(r'[^\u4e00-\u9fffa-zA-Z0-9]+', ' ', text)  # 僅保留中文、英文、數字
#     words = pseg.cut(text)  # 使用 jieba.posseg 進行分詞與詞性標註
#     return " ".join([word for word, flag in words if flag in ['n', 'nz', 'nt', 'nl', 'eng']])  # 保留名詞和專有名詞

# # n：普通名詞
# # 代表一般的名詞，例如「物料」、「廠商」、「現場」。
# # nz：專有名詞
# # 代表專有名詞，例如「SMT」、「Feeder」、「Lenovo」。
# # nt：機構團體名
# # 代表組織或機構的名稱，例如「公司」、「工廠」。
# # nl：地名
# # 代表地點或地名，例如「深圳」、「台北」。
# # eng：英文單詞
# # 代表英文字詞，例如「Rail」、「pin」、「ICT」。

# # v：動詞，例如「操作」、「檢查」。
# # a：形容詞，例如「不良」、「快速」。
# # r：代詞，例如「我們」、「這個」。
# # m：數量詞，例如「一個」、「三次」。
# # d：副詞，例如「非常」、「嚴重」。

# # 清理並分詞
# df['cleaned'] = df['corpus'].progress_apply(clean_and_segment)



In [10]:
input_text = df.loc[340,'corpus']
text = re.sub(r'[^\u4e00-\u9fffa-zA-Z0-9]+', ' ', input_text)  # 僅保留中文、英文、數字
words = pseg.cut(input_text)

for word, flag in words:
    print(word, flag)

1 x
, x
查看 v
不良板 n
， x
確認 v
CPU1 eng
偏位 n
  x
2 x
, x
不良板 n
均 d
為 p
D07B eng
  x
5 x
: x
00 m
- x
5 x
: x
09 m
產出 v
  x
3 x
， x
與 p
在 p
線 n
生技 n
確認 v
為 p
固定 n
相機 n
內 zg
掉落 n
一顆 m
PCI eng
零件 n
導致 v
， x
取出 v
零件 n
後 nr
跟 p
進無 v
不良 a
  x
綜上 n
： x
初步 d
確認 v
為 p
設備異常 n
， x
請 zg
生技 n
及時 c
改善 v
  x
D07 eng
線 n
在 p
生產 n
D12 eng
- x
KYOTO eng
- x
SFF eng
機種 n
時 zg
， x
SMT n
- x
R x
維修 n
區 n
有 v
發現 v
CPU1 eng
偏位 n
不良 a
* x
11 m
PCS eng
， x
不良率 n
為 zg
： x
3.42 m
% x
（ x
11 m
/ x
321 m
） x
， x
還 d
請 v
相關 v
單位 n
協助 v
分析 vn
改善 v
！ x


## TF-IDF 計算

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 1), max_features=1000)  # 可調整 max_features 控制關鍵字數量
tfidf_matrix = vectorizer.fit_transform(df['cleaned'])
keywords = vectorizer.get_feature_names_out()

## 關鍵詞過濾

In [12]:
len(tfidf_matrix.toarray()[0])

1000

In [13]:
len(keywords)

1000

In [74]:
THRESHOLD = 0.2
# 不需要的關鍵詞
BLACKLIST = {
    '綜上', '狀況', '現象',  
    '廠商', '晚班', '對策', '部分', '功能', '時間', '報警', '記錄', '人員', '員工', '新人',
    '問題', '效果', '方向', '參數', '信息', '作業', '現場', '面板', '痕跡', '用量', '規格',
    '物料', '來料', '件數', '結果', '方式', '流程', '環境', '位置', '區域',
    "產生", "led", "dp", "mm", "spec", "外觀", "ok", "t4", "手法", "機種",
    "產線", "立碑", "元件", "cover", "無法", 
    "真空", "parts", "手動", "j210", "qfn", "校正", "環球", "tbl1", "插件", "fail", "ft",
    "pass", "工站", "成品", "過程", "風險", "建議", "pad", "class", "juki", "obe", "全面",
    "插件", "補件", "裝箱", "重點", "開箱", "t3", "高度", "s3", "typec", "用料", "综上", "厂商",
    "定位", "时间", "pth", "實際", "问题", "teach", "夜班", "拋料", "過爐", "sqe", "本體", "参数",
    "时间", "机台", "现场", "用料", "痕迹", "p29a", "厚度", "用料", "tbl2", "p27a", 
    "无法", "pcs", "絲印", "工站", "mm", "um", "情況", "pcs", "夜班", "时间", "t1", "产线",
    "建舜", "成品", "来料", "状况", "高度", "joinsoon", "sqe", "人员", "现场", "痕迹", "錯誤",
    "deren", "Foxconn", "pcie", "slot2", "变形", "定位", "现象", "產生", "接器", "產時",
    "用料", "真空", "重點", "高空", "dc", "pci", "sn", "p29", "環球", "批量", "p27b",
    "本體", "供料", "工程", "線產", "ge", "变形", "对策", "原因", "opt", "作业",
    "神目", "鐵架", "tbl3", "異物", "对策", "過程", "範圍", "j38", "狀態", "音源", "mos", "tbl4",
    "j70", "j71", "dp2", "轨贴", "t2", "p33", "原因", "记录",
    "foxconn", 
    # 新增關鍵詞
    '測試', '校驗', '測量', '工序', '缺陷', '不良', '狀態', '流程卡', '設備', '工藝', '調整', 
    '維護', '人為', '材質', '圖紙', '編號', '分配', '上傳', '標記', '批次', '機型', '檢查', '拆裝', 
    '運行', '清單', '參數調整', '檔案', '庫存', '操作記錄', '工作表', '工藝卡', '制程卡', '版本號', 
    '表面檢查', '焊接', '熱壓', '條碼', '測試結果', '儲存', '零件清單', '生產數據', '技術標準', '設計圖',
    '黑色', '可能性', 'and', '范围', '黑色', '可能性', '科技', '部門', '黑色', '重点',
    '規範', '错误', '部門', '行程', '起子', '嫌疑人', '面積', '动作', '全成信', '人工',
    '個別', '中心', '交接班', '产生', '少件', '嫌疑人', '尺寸', '报警', '过程', '角度',
    'date', 'study', 'me', '生技'
}
    




def filter_keywords(tfidf_scores, keywords, threshold, blacklist):
    filtered_keywords = []
    for score, keyword in zip(tfidf_scores, keywords):
        if score >= threshold and keyword not in blacklist:
            filtered_keywords.append((keyword, score))
    return filtered_keywords


filtered_results = []
for idx, row in enumerate(tfidf_matrix.toarray()):
    filtered = filter_keywords(row, keywords, THRESHOLD, BLACKLIST)
    filtered_results.append(", ".join([f"{kw} ({score:.2f})" for kw, score in filtered]))
    print(f"文本 {idx + 1} 的代表性關鍵詞：")
    for keyword, score in filtered:
        print(f" - {keyword}: {score:.3f}")

文本 1 的代表性關鍵詞：
 - 偏位: 0.244
 - 電感: 0.655
文本 2 的代表性關鍵詞：
 - p31: 0.370
 - trinidad: 0.452
 - 效率: 0.381
 - 直通: 0.584
 - 重影: 0.318
文本 3 的代表性關鍵詞：
 - mac: 0.243
 - sb20t22670: 0.218
 - volga: 0.216
 - 工具: 0.422
 - 撞件: 0.483
 - 端板: 0.249
文本 4 的代表性關鍵詞：
 - l76450: 0.550
 - p37b: 0.497
 - ppid: 0.204
 - tywin: 0.488
 - 產品: 0.235
 - 零件: 0.251
文本 5 的代表性關鍵詞：
 - spi: 0.213
 - 刮刀: 0.259
 - 鋼網: 0.238
 - 錫量: 0.506
文本 6 的代表性關鍵詞：
 - p38: 0.581
 - 燒板: 0.508
 - 維修: 0.210
文本 7 的代表性關鍵詞：
 - juki插件機: 0.238
 - m10197: 0.232
 - p31a: 0.228
 - trinidad: 0.212
 - 壓痕: 0.265
 - 表面: 0.236
 - 電容: 0.237
文本 8 的代表性關鍵詞：
 - buffalo: 0.245
 - d08: 0.224
 - d11: 0.219
 - 剪腳: 0.583
 - 機打: 0.215
 - 腳長: 0.251
 - 自動: 0.237
文本 9 的代表性關鍵詞：
 - m10197: 0.296
 - trinidad: 0.271
 - 反白: 0.267
 - 殘件: 0.611
文本 10 的代表性關鍵詞：
 - dimm2: 0.877
 - ohm: 0.211
 - 阻值: 0.285
文本 11 的代表性關鍵詞：
 - ict: 0.262
 - kent5: 0.208
 - m98850: 0.216
 - pca: 0.234
 - 上線: 0.220
 - 卡板: 0.256
 - 板邊: 0.340
 - 線路: 0.363
 - 軌道: 0.373
文本 12 的代表性關鍵詞：
 - d07: 0.218
 - d10: 

文本 2916 的代表性關鍵詞：
 - p33b: 0.205
 - 全部: 0.428
 - 散熱片: 0.353
 - 生產: 0.208
 - 螺絲: 0.485
文本 2917 的代表性關鍵詞：
 - post: 0.356
 - 混料: 0.586
 - 白色: 0.231
文本 2918 的代表性關鍵詞：
 - bent: 0.205
 - cpu: 0.235
 - f1hc1: 0.207
 - pin: 0.392
文本 2919 的代表性關鍵詞：
 - 刮刀: 0.212
 - 爐前: 0.302
 - 鋼板: 0.276
文本 2920 的代表性關鍵詞：
 - j39: 0.524
文本 2921 的代表性關鍵詞：
 - bat: 0.604
文本 2922 的代表性關鍵詞：
 - card1: 0.639
 - sd: 0.495
 - 變形: 0.245
文本 2923 的代表性關鍵詞：
 - 殘錫: 0.782
文本 2924 的代表性關鍵詞：
 - p32a: 0.551
 - sb20t22670: 0.327
 - volga: 0.324
 - 微鏡: 0.230
 - 缺件: 0.200
文本 2925 的代表性關鍵詞：
 - crf7h: 0.350
 - ict: 0.237
 - p22: 0.278
 - 放板: 0.333
 - 機台: 0.252
 - 鐵件: 0.289
文本 2926 的代表性關鍵詞：
 - ee: 0.209
 - m81628: 0.203
 - oolo: 0.204
 - p32a: 0.234
 - 吸料位置: 0.201
 - 機臺: 0.272
 - 設備貼裝: 0.211
 - 貼裝: 0.395
 - 零件: 0.223
 - 高一: 0.209
文本 2927 的代表性關鍵詞：
 - front: 0.300
 - tc: 0.312
 - 鋼網: 0.780
文本 2928 的代表性關鍵詞：
 - alt: 0.249
 - j94: 0.508
 - l76450: 0.240
 - p35b: 0.217
 - tywin: 0.213
 - 變形: 0.530
 - 鐵殼: 0.275
文本 2929 的代表性關鍵詞：
 - cpu: 0.277
 - 程度: 0.20

 - dimm: 0.255
 - n18258: 0.298
 - operationr: 0.297
 - p27: 0.249
 - xmm1: 0.398
 - 原物料: 0.226
 - 焊點: 0.247
 - 點位: 0.264
文本 5811 的代表性關鍵詞：
 - akalir: 0.242
 - n25948: 0.248
 - pqe: 0.229
 - 空焊: 0.414
 - 處理: 0.212
 - 隱患: 0.230
文本 5812 的代表性關鍵詞：
 - cres: 0.266
 - hsbp: 0.284
 - p21b: 0.271
 - 吸嘴: 0.382
 - 殘錫: 0.204
 - 缺件: 0.528
 - 高一: 0.216
文本 5813 的代表性關鍵詞：
 - p6y94: 0.205
 - usb3s3: 0.428
 - 短路: 0.366
 - 鋼板: 0.441
文本 5814 的代表性關鍵詞：
 - 上料員: 0.211
 - 料帶: 0.223
 - 週期: 0.241
 - 高五: 0.207
文本 5815 的代表性關鍵詞：
 - cpu: 0.412
 - driver: 0.252
 - dsw: 0.223
 - p39a: 0.241
 - s0: 0.220
 - short: 0.320
 - vcore: 0.221
 - vtky7: 0.231
 - 殘錫: 0.389
文本 5816 的代表性關鍵詞：
 - dimm: 0.372
 - grs: 0.269
 - input: 0.249
 - lotes: 0.244
 - rplr: 0.228
 - ytm400: 0.235
 - 折角: 0.567
文本 5817 的代表性關鍵詞：
 - cpu: 0.284
 - n18252: 0.403
 - scrabbler: 0.413
 - 主板: 0.294
 - 品質: 0.340
 - 重影: 0.331
文本 5818 的代表性關鍵詞：
 - cpu: 0.327
 - 模組: 0.213
 - 機臺: 0.214
 - 異常: 0.218
 - 破損: 0.259
文本 5819 的代表性關鍵詞：
 - dominor: 0.436
 - n18256: 0.44

In [75]:
df['extracted_keywords'] = filtered_results

## 輸出結果

In [76]:
def clean_tfidf_keywords(keyword_string):
    # 分隔關鍵詞，提取括號前的部分
    keywords = [item.split(' (')[0] for item in keyword_string.split(',') if item.strip()]
    # 使用空格拼接關鍵詞
    return ' '.join(keywords)


df['tfidf_keywords'] = df['extracted_keywords'].apply(clean_tfidf_keywords)

In [77]:
df[df['bill_no'] == 'PDPC20230707004']

,bill_no,group_line,manufacture_time,exception_desc,defects_name,reason_fix,qc_suggestion,corpus,cleaned,extracted_keywords,tfidf_keywords
4038,PDPC20230707004,P21,2023/7/7,P21B線在生產UNELLA-ALT（L85069-002）機種時，SMT維修發現KL39反...,反白缺件,1.確認不良為KL39反白缺件不良；_x000D_\n2.查詢不良路由時間為7/7 07:3...,更換Feeder後，跟進2H(7/7 9:00-11:00)無相同不良產出,1.確認不良為KL39反白缺件不良； 2.查詢不良路由時間為7/7 07:37-08:32產...,1 KL39 反白 缺件 2 路由 時間 7 3 高...,"alt (0.27), feeder (0.39), p21b (0.28), 反白 (0....",alt feeder p21b 反白 缺件 貼裝坐標 高三


In [79]:
df.loc[107, 'cleaned']

df.loc[107, 'extracted_keywords']

'd08 (0.24), x0y8 (0.62), 吸嘴 (0.40), 殘錫 (0.21), 缺件 (0.37), 電阻 (0.22)'

In [80]:
# 將所有單詞收集到一個列表中，排除空字串，並去重
unique_words = set(
    word
    for keywords in df['tfidf_keywords']
    if isinstance(keywords, str)  # 確保是字串
    for word in keywords.split()  # 用空白分割
)

# 將結果轉回有序列表（可選）
unique_words = sorted(unique_words)

print(unique_words)

['0201電容', '0402電容', 'a1', 'a6', 'a8', 'a9', 'ab', 'act', 'adl', 'agp', 'ah00', 'ai', 'aj76', 'aj78', 'akali', 'akalir', 'alexandria', 'alt', 'alw', 'amd', 'aoi', 'aos', 'au14', 'aud', 'audio', 'audio1', 'aux', 'b1', 'bastion', 'bat', 'bb4', 'bent', 'bga', 'bios', 'blizzard', 'bohemia', 'bom', 'bti', 'buffalo', 'c1', 'c7', 'ca', 'card', 'card1', 'cersei', 'cezanne', 'cn1', 'con', 'controller', 'cpu', 'cpu0', 'cpu1', 'cr10', 'cr14', 'cr15', 'cres', 'crf7h', 'cxr46', 'd04', 'd05', 'd07', 'd08', 'd09', 'd10', 'd10a', 'd10b', 'd11', 'd12', 'd13', 'd14', 'd2502', 'dali', 'dc48c', 'dell', 'dev', 'dimm', 'dimm1', 'dimm2', 'dimm3', 'dimm4', 'dmyn9', 'dolphin', 'domino', 'dominor', 'dp1', 'dp3', 'dp浮高', 'drive', 'driver', 'dsw', 'dual', 'dy62r', 'e3', 'e49', 'ec', 'ec1', 'ee', 'ej31', 'eldorado', 'elite', 'er', 'esd', 'exp', 'f1', 'f1hc1', 'f209', 'f284y', 'f4y1m', 'fa', 'failed', 'faillocation', 'fan', 'fbt', 'feeder', 'ff', 'ff1', 'ff2', 'ff3', 'ff4', 'fg47g', 'final', 'fio', 'fit', 'freeport

In [82]:
df[['corpus','tfidf_keywords']].head(20)

,corpus,tfidf_keywords
0,1.現場確認為電感偏位不良，3顆用量，3個位置均有不良產出； 2.查詢不良板路由時間：1/7...,偏位 電感
1,P31線生產Trinidad，ICT直通率35%左右，嚴重影響生產效率，請幫忙協助解決，謝謝！,p31 trinidad 效率 直通 重影
2,跟進狀況： 1>Lenovo WS機種，SMT貼裝0402電容，反面板邊位置； 2>確認不良...,mac sb20t22670 volga 工具 撞件 端板
3,OBE抽檢P37B TYWIN（L76450-002）產品，發現MR206&MR206零件燒...,l76450 p37b ppid tywin 產品 零件
4,"1.顯微鏡下觀察，不良板功能Pin根部少錫不良（單個功能Pin腳焊盤中間存在""斷層""現象，錫...",spi 刮刀 鋼網 錫量
5,P38線生產IB460CX-M430C-38S，FT測試發現不開機 送FT維修分析燒板 請協...,p38 燒板 維修
6,"跟進狀況： 1、確認不良板外觀PCE230(820uF 2.5V電容)正面浮高,反面腳未出,...",juki插件機 m10197 p31a trinidad 壓痕 表面 電容
7,跟進狀況 1>PTH機打電容，目視零件單邊腳未剪斷（有剪切痕跡）； 2>神目無法Cover此...,buffalo d08 d11 剪腳 機打 腳長 自動
8,跟進狀況 1>確認Q45位置三極管不貼件，目視殘10PIN QFN反白，確認AOI無法Cov...,m10197 trinidad 反白 殘件
9,1>確認DIMM2不開機； 2>直接DIMM2量測到DIMM2-148對地無阻值，DIMM2...,dimm2 ohm 阻值


In [81]:
df.to_excel('PDCS_TFIDF.xlsx', index=False)

## Apriori

In [83]:
import pandas as pd


df = pd.read_excel('PDCS_TFIDF.xlsx')


df['defects_name'] = df['defects_name'].fillna('')
df_tmp = df[df['defects_name'].str.contains('缺件')].reset_index(drop=True).copy()

In [84]:
df_tmp[df_tmp['bill_no'] == 'PDPC20230707004']

,bill_no,group_line,manufacture_time,exception_desc,defects_name,reason_fix,qc_suggestion,corpus,cleaned,extracted_keywords,tfidf_keywords
117,PDPC20230707004,P21,2023/7/7,P21B線在生產UNELLA-ALT（L85069-002）機種時，SMT維修發現KL39反...,反白缺件,1.確認不良為KL39反白缺件不良；_x000D_\n2.查詢不良路由時間為7/7 07:3...,更換Feeder後，跟進2H(7/7 9:00-11:00)無相同不良產出,1.確認不良為KL39反白缺件不良； 2.查詢不良路由時間為7/7 07:37-08:32產...,1 KL39 反白 缺件 2 路由 時間 7 3 高...,"alt (0.27), feeder (0.39), p21b (0.28), 反白 (0....",alt feeder p21b 反白 缺件 貼裝坐標 高三


In [85]:
# Step 1: 過濾掉空字串
df_tmp = df[df['tfidf_keywords'].str.strip() != ''].copy()

# Step 2: 將每個非空字串以空白分割，轉換成詞列表
df_tmp['tfidf_keywords'] = df_tmp['tfidf_keywords'].fillna('')
df_tmp['tfidf_keywords'] = df_tmp['tfidf_keywords'].apply(lambda x: x.split())


transactions = df_tmp['tfidf_keywords'].tolist()
transactions = [t for t in transactions if t]

In [86]:
print(len(transactions))
transactions 

#相機

7296


[['偏位', '電感'],
 ['p31', 'trinidad', '效率', '直通', '重影'],
 ['mac', 'sb20t22670', 'volga', '工具', '撞件', '端板'],
 ['l76450', 'p37b', 'ppid', 'tywin', '產品', '零件'],
 ['spi', '刮刀', '鋼網', '錫量'],
 ['p38', '燒板', '維修'],
 ['juki插件機', 'm10197', 'p31a', 'trinidad', '壓痕', '表面', '電容'],
 ['buffalo', 'd08', 'd11', '剪腳', '機打', '腳長', '自動'],
 ['m10197', 'trinidad', '反白', '殘件'],
 ['dimm2', 'ohm', '阻值'],
 ['ict', 'kent5', 'm98850', 'pca', '上線', '卡板', '板邊', '線路', '軌道'],
 ['d07', 'd10', 'ssd', '短路'],
 ['d05', 'k5c', '上線', '壓件', '螺絲'],
 ['pin', 'rt', 'smt', 'x0y8', '結構'],
 ['j39', 'jamie', 'l76451', 'p35b', '微鏡', '膠紙'],
 ['p3d', 'tywin', '條件', '雙面', '電容'],
 ['final', 'vi', '偏位', '腳座', '貼裝頭', '電容'],
 ['cpu', 'p34b', 'pin', '堆板', '幹部', '背板', '螺絲'],
 ['pin', 'rt', '折角'],
 ['p34b', '異常', '電容'],
 ['d05', 'dimm', 'dimm1', 'jwjy', '技能', '插件機', '數量', '目檢'],
 ['孔徑', '直徑'],
 ['m09983', 'pike', 's31', '偏位', '電感'],
 ['hu', 'pe', '板子', '花板', '設計', '變形', '貼片機'],
 ['aoi', 'spi', '偏位', '錫膏印刷', '錫量'],
 ['p34a', '幹部', '技能', '撞件', '

In [134]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


# 將交易資料轉換為 one-hot 編碼的 DataFrame
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

# 使用 Apriori 找出高頻項目集
# min_support 可調整，例如設定為0.2，代表至少出現在 20% 交易中的項目組   0.004 = 421 至少 2 筆  0.00237=1
frequent_itemsets = apriori(df_encoded, min_support=0.0009, use_colnames=True)

print("高頻項目集 (Frequent Itemsets):")
frequent_itemsets

高頻項目集 (Frequent Itemsets):


,support,itemsets
0,0.005482,(0201電容)
1,0.015899,(0402電容)
2,0.003701,(a1)
3,0.006990,(a6)
4,0.003701,(a8)
...,...,...
5758,0.000959,"(act, loc, dev, msr, a1, lm, std, lo, name, step)"
5759,0.000959,"(act, loc, msr, hi, a1, lm, std, lo, name, step)"
5760,0.000959,"(loc, dev, msr, hi, a1, lm, std, lo, name, step)"
5761,0.001371,"(act, loc, dev, msr, hi, lm, std, lo, name, step)"


In [135]:
# 支持度 (support)

# 定義：高頻項目出現的比例。對於稀疏數據，設置較低的 support（如 0.01 至 0.05）更適合，以避免過高的門檻篩掉有意義的規則。
# 計算範例：支持度 0.01 表示項目需出現在至少 421 * 0.01 ≈ 4 筆交易中。

# 置信度 (confidence)

# 定義：當條件成立時，結果成立的概率。較低的 confidence（如 0.5 至 0.7）可生成更多規則，但可能包含較多低質量規則。
# 如果規則質量重要，可設置較高的 confidence（如 0.7 至 0.9）。

In [140]:
# 根據高頻項目集產生關聯規則
# metric 可以選 'confidence'、'lift'、'leverage'等，min_threshold 可依需要調整
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.007)

columns_to_round = [
    'antecedent support', 'consequent support', 'support', 
    'confidence', 'lift', 'leverage', 'conviction', 'zhangs_metric'
]
rules[columns_to_round] = rules[columns_to_round].round(4)


### **欄位解釋**

| **欄位名稱**            | **解釋**                                                                                                    |
|-------------------------|-----------------------------------------------------------------------------------------------------------|
| **`antecedents`**       | 前件項目（規則的 "如果" 部分）。                                                                          |
| **`consequents`**       | 後件項目（規則的 "那麼" 部分）。                                                                          |
| **`antecedent support`**| 前件項目出現的支持度，表示前件項目在所有交易中的比例。                                                       |
| **`consequent support`**| 後件項目出現的支持度，表示後件項目在所有交易中的比例。                                                       |
| **`support`**           | 前件和後件項目同時出現的支持度，表示兩者同時出現在交易中的比例。                                            |
| **`confidence`**        | 置信度，表示在包含前件項目的交易中，後件項目出現的比例。計算公式：`support / antecedent support`           |
| **`lift`**              | 提升度，表示前件和後件項目之間的相關性強度。計算公式：`confidence / consequent support`                   |
| **`leverage`**          | 杠杆值，表示前後項目同時出現的實際頻率與預期頻率的差異。計算公式：`support - (antecedent support × consequent support)` |
| **`conviction`**        | 信念度，衡量規則的可靠性。計算公式：`(1 - consequent support) / (1 - confidence)`                          |
| **`zhangs_metric`**     | 張氏指標，衡量規則的重要性，介於 -1 和 1，值越接近 1 表示關聯越強。                                        |


In [150]:
print("\n關聯規則 (Association Rules):")
rules = rules[['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'zhangs_metric']]#.head(30)


關聯規則 (Association Rules):


In [142]:
rules.head(80)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(偏位),(0201電容),0.1028,0.0055,0.0022,0.0213,3.8912,0.0016,1.0162,0.8281
1,(0201電容),(偏位),0.0055,0.1028,0.0022,0.4000,3.8912,0.0016,1.4953,0.7471
2,(0201電容),(吸嘴),0.0055,0.0232,0.0012,0.2250,9.7136,0.0011,1.2604,0.9020
3,(吸嘴),(0201電容),0.0232,0.0055,0.0012,0.0533,9.7136,0.0011,1.0505,0.9183
4,(撞件),(0201電容),0.0536,0.0055,0.0010,0.0179,3.2655,0.0007,1.0126,0.7331
...,...,...,...,...,...,...,...,...,...,...
75,(a1),(name),0.0037,0.0023,0.0011,0.2963,127.1634,0.0011,1.4177,0.9958
76,(std),(a1),0.0022,0.0037,0.0011,0.5000,135.1111,0.0011,1.9926,0.9948
77,(a1),(std),0.0037,0.0022,0.0011,0.2963,135.1111,0.0011,1.4179,0.9963
78,(step),(a1),0.0026,0.0037,0.0014,0.5263,142.2222,0.0014,2.1033,0.9956


In [143]:
rules[(rules['antecedents'] == {'吸嘴'})&(rules['consequents'] == {'殘錫'})].reset_index(drop=True)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(吸嘴),(殘錫),0.0232,0.0203,0.0029,0.1243,6.1257,0.0024,1.1187,0.8566


In [148]:
rules[(rules['antecedents'] == {'吸嘴'})].reset_index(drop=True)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(吸嘴),(0201電容),0.0232,0.0055,0.0012,0.0533,9.7136,0.0011,1.0505,0.9183
1,(吸嘴),(0402電容),0.0232,0.0159,0.0012,0.0533,3.3495,0.0009,1.0395,0.7181
2,(吸嘴),(high),0.0232,0.0108,0.0010,0.0414,3.8253,0.0007,1.0319,0.7561
3,(吸嘴),(kyoto),0.0232,0.0111,0.0011,0.0473,4.2639,0.0008,1.0380,0.7836
4,(吸嘴),(p29b),0.0232,0.0055,0.0011,0.0473,8.6343,0.0010,1.0439,0.9051
5,(吸嘴),(ven),0.0232,0.0104,0.0011,0.0473,4.5444,0.0009,1.0388,0.7984
6,(吸嘴),(保養),0.0232,0.0070,0.0030,0.1302,18.6230,0.0029,1.1416,0.9687
7,(吸嘴),(偏位),0.0232,0.1028,0.0079,0.3432,3.3386,0.0056,1.3660,0.7171
8,(吸嘴),(吸料位置),0.0232,0.0145,0.0011,0.0473,3.2582,0.0008,1.0344,0.7095
9,(吸嘴),(殘錫),0.0232,0.0203,0.0029,0.1243,6.1257,0.0024,1.1187,0.8566


In [144]:
rules[(rules['antecedents'] == {'反白'})].reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(反白),(aoi),0.0129,0.0270,0.0010,0.0745,2.7580,0.0006,1.0513,0.6457
1,(反白),(feeder),0.0129,0.0210,0.0070,0.5426,25.8723,0.0067,2.1402,0.9739
2,(反白),(ic),0.0129,0.0199,0.0010,0.0745,3.7470,0.0007,1.0590,0.7427
3,(反白),(y5),0.0129,0.0012,0.0011,0.0851,68.9929,0.0011,1.0917,0.9984
4,(反白),(供料不穩),0.0129,0.0099,0.0032,0.2447,24.7943,0.0030,1.3109,0.9722
5,(反白),(偏位),0.0129,0.1028,0.0014,0.1064,1.0349,0.0000,1.0040,0.0342
6,(反白),(備料),0.0129,0.0022,0.0012,0.0957,43.6596,0.0012,1.1035,0.9898
7,(反白),(晶振),0.0129,0.0025,0.0014,0.1064,43.1206,0.0013,1.1163,0.9896
8,(反白),(缺件),0.0129,0.0387,0.0010,0.0745,1.9267,0.0005,1.0387,0.4872
9,(反白),(高四),0.0129,0.0112,0.0018,0.1383,12.3051,0.0016,1.1475,0.9307


In [145]:
rules[(rules['antecedents'] == {'反白'})&(rules['consequents'] == {'缺件'})].reset_index(drop=True)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(反白),(缺件),0.0129,0.0387,0.001,0.0745,1.9267,0.0005,1.0387,0.4872


In [146]:
rules[rules['antecedents'] == {'feeder'}].reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(feeder),(aoi),0.021,0.0270,0.0011,0.0523,1.9365,0.0005,1.0267,0.4940
1,(feeder),(p25),0.021,0.0136,0.0010,0.0458,3.3718,0.0007,1.0337,0.7185
2,(feeder),(repair),0.021,0.0108,0.0012,0.0588,5.4326,0.0010,1.0510,0.8334
3,(feeder),(s31),0.021,0.0048,0.0011,0.0523,10.8997,0.0010,1.0501,0.9277
4,(feeder),(供料不穩),0.021,0.0099,0.0086,0.4118,41.7255,0.0084,1.6832,0.9969
5,(feeder),(偏位),0.021,0.1028,0.0064,0.3072,2.9883,0.0043,1.2950,0.6796
6,(feeder),(反白),0.021,0.0129,0.0070,0.3333,25.8723,0.0067,1.4807,0.9819
7,(feeder),(吸料位置),0.021,0.0145,0.0011,0.0523,3.5990,0.0008,1.0398,0.7376
8,(feeder),(墊片),0.021,0.0025,0.0011,0.0523,21.1939,0.0010,1.0526,0.9732
9,(feeder),(缺件),0.021,0.0387,0.0027,0.1307,3.3820,0.0019,1.1059,0.7194


In [147]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(偏位),(0201電容),0.1028,0.0055,0.0022,0.0213,3.8912,0.0016,1.0162,0.8281
1,(0201電容),(偏位),0.0055,0.1028,0.0022,0.4000,3.8912,0.0016,1.4953,0.7471
2,(0201電容),(吸嘴),0.0055,0.0232,0.0012,0.2250,9.7136,0.0011,1.2604,0.9020
3,(吸嘴),(0201電容),0.0232,0.0055,0.0012,0.0533,9.7136,0.0011,1.0505,0.9183
4,(撞件),(0201電容),0.0536,0.0055,0.0010,0.0179,3.2655,0.0007,1.0126,0.7331
...,...,...,...,...,...,...,...,...,...,...
185485,(lm),"(act, loc, dev, msr, hi, a1, std, lo, name, step)",0.0217,0.0010,0.0010,0.0443,46.1772,0.0009,1.0454,1.0000
185486,(std),"(act, loc, dev, msr, hi, a1, lm, lo, name, step)",0.0022,0.0010,0.0010,0.4375,456.0000,0.0010,1.7761,1.0000
185487,(lo),"(act, loc, dev, msr, hi, a1, lm, std, name, step)",0.0019,0.0010,0.0010,0.5000,521.1429,0.0010,1.9981,1.0000
185488,(name),"(act, loc, dev, msr, hi, a1, lm, std, lo, step)",0.0023,0.0010,0.0010,0.4118,429.1765,0.0010,1.6984,1.0000


In [153]:
def format_frozenset_columns(df, columns):
    """
    Format frozenset columns in a DataFrame to remove the frozenset wrapper.

    Args:
        df (pd.DataFrame): The DataFrame containing frozenset columns.
        columns (list): List of column names to format.

    Returns:
        pd.DataFrame: The formatted DataFrame.
    """
    df = df.copy()
    for column in columns:
        df[column] = df[column].apply(lambda x: ', '.join(x) if isinstance(x, frozenset) else x)
    return df



formatted_rules = format_frozenset_columns(rules, ["antecedents", "consequents"])

In [151]:
rules.to_excel('PDCS_Rules.xlsx', index=False)